[IPython Notebook](20_21_meshes.ipynb) |  [Python Script](20_21_meshes.py)

2.0 - 2.1 Migration: Meshes
============================

In [1]:
import phoebe

PHOEBE 2.0 had a mesh dataset along with `pbmesh` and `protomesh` options you could send to `b.run_compute`.  These options were quite convenient, but had a few inherit problems:

* The protomesh was exposed at only t0 and was in roche coordinates, despite using the same 'x', 'y', 'z' qualifiers.
* Passband-dependent Parameters were exposed for the mesh if `pbmesh=True`, but only if the times matched exactly with the passband (lc, rv, etc) dataset.
* Storing more than a few meshes become very memory intensive due to their large size and the large number of columns.

Addressing these shortcomings required a complete redesign of the mesh dataset.  The most important changes are the following:

* `pbmesh` and `protomesh` are no longer valid options.  Everything is done through the mesh dataset itself.
* The `columns` Parameter in the mesh dataset lists the extra columns that will be exposed in the model mesh when calling `run_compute`.  By default only the elements (in both roche and plane-of-sky) are returned (and these columns cannot be disabled).  See the section on columns below for more details.
* Included in the `columns` Parameter, you can choose whether to expose coordinates in the Roche system (xyz) or the plane-of-sky system (uvw).  When plotting, the defaults remain plane-of-sky, but will now see plotting in uvw, whereas in PHOEBE 2.0.x these were still called xyz.  Note that this also applies to velocities (vxs, vys, vzs vs vus, vvs, vws).
* The `include_times` Parameter allows for importing times (at `run_compute` time) from other datasets or various t0s.  In this way, you can include a mesh for all times in your light curve without worrying about the arrays getting out-of-sync.
* By default, as with all datasets now, the `times` Parameter is empty.  If you do not set `times` or `include_times` before calling `run_compute`, your model will be empty.

### columns

The `columns` Parameter is a SelectParameter (a new type of Parameter introduced in PHOEBE 2.1), meaning its value is a list of strings, each of which must be a valid entry.  You can find the list of entries by calling `param.get_choices()` (same as you would for a ChoiceParameter).  Additionally, the value accepts wildcards, so long as at least one match is made with the choices.  This allows you to easily select `rvs` from all datasets, for example.

As an example let's add a few datasets and look at the available choices for the `columns` Parameter.

In [2]:
b = phoebe.default_binary()
b.add_dataset('mesh')
print b.get_parameter('columns').get_choices()

['pot', 'rpole', 'volume', 'xs', 'ys', 'zs', 'vxs', 'vys', 'vzs', 'nxs', 'nys', 'nzs', 'us', 'vs', 'ws', 'vus', 'vvs', 'vws', 'nus', 'nvs', 'nws', 'areas', 'loggs', 'teffs', 'rprojs', 'mus', 'visibilities', 'visible_centroids', 'rs']


In [3]:
b.add_dataset('lc')
print b.get_parameter('columns').get_choices()

['pot', 'rpole', 'volume', 'xs', 'ys', 'zs', 'vxs', 'vys', 'vzs', 'nxs', 'nys', 'nzs', 'us', 'vs', 'ws', 'vus', 'vvs', 'vws', 'nus', 'nvs', 'nws', 'areas', 'loggs', 'teffs', 'rprojs', 'mus', 'visibilities', 'visible_centroids', 'rs', 'intensities@lc01', 'normal_intensities@lc01', 'abs_intensities@lc01', 'abs_normal_intensities@lc01', 'boost_factors@lc01', 'ldint@lc01', 'pblum@lc01', 'ptfarea@lc01']


In [6]:
b['columns'] = ['*@lc01', 'teffs']
b.get_parameter('columns').get_value()

['*@lc01', 'teffs']

In [7]:
b.get_parameter('columns').expand_value()

['intensities@lc01',
 'normal_intensities@lc01',
 'abs_intensities@lc01',
 'abs_normal_intensities@lc01',
 'boost_factors@lc01',
 'ldint@lc01',
 'pblum@lc01',
 'ptfarea@lc01',
 'teffs']

### include_times

Similarly, the `include_times` Parameter is a SelectParameter, with the choices being the existing datasets as well as various t0s.

In [8]:
b = phoebe.default_binary()
b.add_dataset('mesh')
print b.get_parameter('include_times').get_value()

[]


In [9]:
print b.get_parameter('include_times').get_choices()

['t0_perpass@binary', 't0_supconj@binary', 't0_ref@binary', 't0@system']


In [10]:
b.add_dataset('lc')
print b.get_parameter('include_times').get_choices()

['lc01', 't0_perpass@binary', 't0_supconj@binary', 't0_ref@binary', 't0@system']


In [11]:
b['include_times'] = ['lc01', 't0@system']

In [12]:
print b.get_parameter('include_times').get_value()

['lc01', 't0@system']
